In [1]:
from __future__ import print_function

import keras
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras import layers

import matplotlib.pyplot as plt

import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 50


In [3]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


404 train samples
102 test samples


In [4]:
# preparing data
x_train_mean = x_train.mean(axis=0)
print(f'x_train mean: {x_train_mean}')
x_train = x_train - x_train_mean

x_train_std = x_train.std(axis=0)
print(f'x_train_std: {x_train_std}')
x_train = x_train / x_train_std

x_test = x_test - x_train_mean
x_test = x_test / x_train_std
print(x_train[0])


x_train mean: [3.74511057e+00 1.14801980e+01 1.11044307e+01 6.18811881e-02
 5.57355941e-01 6.26708168e+00 6.90106436e+01 3.74027079e+00
 9.44059406e+00 4.05898515e+02 1.84759901e+01 3.54783168e+02
 1.27408168e+01]
x_train_std: [9.22929073e+00 2.37382770e+01 6.80287253e+00 2.40939633e-01
 1.17147847e-01 7.08908627e-01 2.79060634e+01 2.02770050e+00
 8.68758849e+00 1.66168506e+02 2.19765689e+00 9.39946015e+01
 7.24556085e+00]
[-0.27224633 -0.48361547 -0.43576161 -0.25683275 -0.1652266  -0.1764426
  0.81306188  0.1166983  -0.62624905 -0.59517003  1.14850044  0.44807713
  0.8252202 ]


In [ ]:
def build_model(x):
    inputs = keras.Input(shape=(13,))
    x = layers.Dense(13, activation='relu', input_dim=13)(inputs)
    x = layers.Dense(13, activation='relu')(x)
    outputs = layers.Dense(1, activation='relu')(x)
    
    return keras.Model(inputs, outputs)


In [ ]:
model_1 = build_model(x_train)
model_1.compile(optimizer='rmsprop', loss='mse',metrics=['mae'])
history = model_1.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))


score = model_1.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Mean absolute error:', score[1])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 404 samples, validate on 102 samples
Epoch 1/50
404/404 [==============================] - 0s 552us/step - loss: 565.8116 - mean_absolute_error: 21.9850 - val_loss: 586.7310 - val_mean_absolute_error: 22.5187
Epoch 2/50
404/404 [==============================] - 0s 13us/step - loss: 558.5703 - mean_absolute_error: 21.8242 - val_loss: 579.7244 - val_mean_absolute_error: 22.3700
Epoch 3/50
404/404 [==============================] - 0s 21us/step - loss: 552.3072 - mean_absolute_error: 21.6825 - val_loss: 573.4131 - val_mean_absolute_error: 22.2277
Epoch 4/50
404/404 [==============================] - 0s 17us/step - loss: 546.5915 - mean_absolute_error: 21.5480 - val_loss: 567.5785 - val_mean_absolute_error: 22.0946
Epoch 5/50
404/404 [==============================] - 0s 27us/step - loss: 541.1745 - mean_absolute_error: 21.4196 - val_loss: 561.4198 - val_mean_ab

404/404 [==============================] - 0s 26us/step - loss: 212.2155 - mean_absolute_error: 12.4889 - val_loss: 209.1505 - val_mean_absolute_error: 12.4076
Epoch 46/50
404/404 [==============================] - 0s 22us/step - loss: 202.2709 - mean_absolute_error: 12.1577 - val_loss: 199.6005 - val_mean_absolute_error: 12.0880
Epoch 47/50
404/404 [==============================] - 0s 31us/step - loss: 192.9377 - mean_absolute_error: 11.8402 - val_loss: 190.3585 - val_mean_absolute_error: 11.7650
Epoch 48/50
404/404 [==============================] - 0s 14us/step - loss: 183.7388 - mean_absolute_error: 11.5149 - val_loss: 179.8740 - val_mean_absolute_error: 11.3874
Epoch 49/50
404/404 [==============================] - 0s 32us/step - loss: 173.6676 - mean_absolute_error: 11.1575 - val_loss: 170.8584 - val_mean_absolute_error: 11.0496
Epoch 50/50
404/404 [==============================] - 0s 22us/step - loss: 164.8214 - mean_absolute_error: 10.8335 - val_loss: 161.7685 - val_mean_abso

In [ ]:
# K-Fold
k = 10

num_val_samples = len(x_train) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print(f'Processing fold # {i}')
    val_data = x_train[i * num_val_samples: (i+1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate(
                            [x_train[:i * num_val_samples],
                            x_train[(i+1) * num_val_samples:]],
                            axis=0)
    partial_train_targets = np.concatenate(
                            [y_train[:i * num_val_samples],
                            y_train[(i+1)*num_val_samples:]],
                            axis=0)
    
    model = build_model(val_data)
    model.compile(optimizer='rmsprop', loss='mse',metrics=['mae'])
    model.fit(partial_train_data,
              partial_train_targets,
              epochs=num_epochs,
              batch_size=1,
              verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

Processing fold # 0
Processing fold # 1
Processing fold # 2
Processing fold # 3
Processing fold # 4
Processing fold # 5
Processing fold # 6
Processing fold # 7


In [ ]:
print(f'all_scores : {all_scores}')
print(f'mean all scores : {np.mean(all_scores)}')